In [1]:
import torch
import torch.nn as nn

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DrivingModel(nn.Module):
    def __init__(self):
        super(DrivingModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 24, kernel_size=5, padding=2, stride=2)
        self.conv2 = nn.Conv2d(24, 36, kernel_size=5, padding=2, stride=2)
        self.conv3 = nn.Conv2d(36, 48, kernel_size=3, padding=1, stride=2)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=3, padding=1, stride=1)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, padding=1, stride=1)

        # π projection
        self.projection = nn.Linear(64 * 10 * 20, 64 * 10)
        
        self.lstm = nn.LSTM(64 * 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))

        return x

input = torch.randn(1, 3, 80, 160)

model = DrivingModel()

output = model(input)




torch.Size([1, 24, 40, 80])
torch.Size([1, 36, 20, 40])
torch.Size([1, 48, 10, 20])
torch.Size([1, 64, 10, 20])
torch.Size([1, 64, 10, 20])


In [5]:
from torchvision.models import resnet50, ResNet50_Weights

backbone = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

print(backbone)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:09<00:00, 10.4MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
import torch
import torch.nn as nn

class FPN(nn.Module):
    def __init__(self, backbone, out_channels, n_classes):
        super(FPN, self).__init__()
        self.backbone = backbone

        self.upsample = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2)
        self.conv1x1 = nn.Conv2d(out_channels, out_channels, kernel_size=1)
        self.conv3x3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

        self.fc_classification = nn.Sequential(
            nn.Flatten(),
            nn.Linear(out_channels * 224 * 224, 1024),
            nn.ReLU(),
            nn.Linear(1024, n_classes)
        )

        self.fc_regression = nn.Sequential(
            nn.Flatten(),
            nn.Linear(out_channels * 224 * 224, 1024),
            nn.ReLU(),
            nn.Linear(1024, 4)
        )

    def forward(self, stage1):
        stage2 = self.backbone.layer1(stage1)
        stage3 = self.backbone.layer2(stage2)
        stage4 = self.backbone.layer3(stage3)

        C4 = self.upsample(stage4)
        C3 = self.conv1x1(stage3) + self.upsample(C4)
        C2 = self.conv1x1(stage2) + self.upsample(C3)

        P4 = self.conv3x3(C4)
        P3 = self.conv3x3(C3)
        P2 = self.conv3x3(C2)

        # Aplanar y pasar por las capas fully connected
        classification_p4 = self.fc_classification(P4)
        regression_p4 = self.fc_regression(P4)

        classification_p3 = self.fc_classification(P3)
        regression_p3 = self.fc_regression(P3)

        classification_p2 = self.fc_classification(P2)
        regression_p2 = self.fc_regression(P2)

        return {
            'P2': {'classification': classification_p2, 'regression': regression_p2},
            'P3': {'classification': classification_p3, 'regression': regression_p3},
            'P4': {'classification': classification_p4, 'regression': regression_p4},
        }
        

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, moment